<a href="https://colab.research.google.com/github/matsunagalab/mdzen/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MDZen: AI-Powered Molecular Dynamics Agent

**Interactive AI assistant for setting up MD simulations**

## Workflow

1. **Setup** - Install dependencies (Konda + AmberTools)
2. **Phase 1: Clarification** - Describe your simulation, AI generates SimulationBrief
3. **Edit Brief** - Review and customize simulation parameters
4. **Phase 2: Execute** - Run workflow step by step
5. **Visualization** - View trajectory animation with py3Dmol
6. **Download** - Get all generated files

---

## Quick Start

1. Set **API key** in Colab secrets (ANTHROPIC_API_KEY, OPENAI_API_KEY, or GOOGLE_API_KEY)
2. Run **Setup** cell (~5-10 min)
3. Run **Phase 1** - describe your simulation
4. **Edit Brief** - customize parameters if needed
5. Run **Phase 2** - click buttons for each step
6. **Visualize** and **Download** results

---
## Setup: Install Konda and Dependencies

**Konda** is a simple wrapper for conda in Google Colab.
- No kernel restart needed (unlike condacolab)
- Uses conda for package installation
- Installation takes ~5-10 minutes

**API Key**: Set one of the following in Colab secrets:
- `ANTHROPIC_API_KEY` for Claude
- `OPENAI_API_KEY` for GPT-4
- `GOOGLE_API_KEY` for Gemini

In [ ]:
import sys
import os
import time

IN_COLAB = 'google.colab' in sys.modules

# ============================================================================
# Detect and set API keys from Colab secrets
# ============================================================================
detected_provider = None

if IN_COLAB:
    from google.colab import userdata
    
    # Try to detect API keys from Colab secrets
    api_keys = {
        'ANTHROPIC_API_KEY': 'anthropic',
        'OPENAI_API_KEY': 'openai',
        'GOOGLE_API_KEY': 'google',
    }
    
    for key_name, provider in api_keys.items():
        try:
            key_value = userdata.get(key_name)
            if key_value:
                os.environ[key_name] = key_value
                if detected_provider is None:
                    detected_provider = provider
                print(f"✓ {key_name} loaded from Colab secrets ({provider})")
        except:
            pass
    
    if detected_provider is None:
        print("⚠️ WARNING: No API key found in Colab secrets!")
        print("Please add one of: ANTHROPIC_API_KEY, OPENAI_API_KEY, or GOOGLE_API_KEY")
        print("Go to: Settings (gear icon) > Secrets")
    else:
        print(f"\n🤖 Using {detected_provider.upper()} as LLM provider")
else:
    # Local - check environment variables
    for key_name in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'GOOGLE_API_KEY']:
        if os.environ.get(key_name):
            detected_provider = key_name.split('_')[0].lower()
            print(f"✓ {key_name} found in environment ({detected_provider})")
            break

# ============================================================================
# Install dependencies
# ============================================================================
if IN_COLAB:
    start_time = time.time()

    # Install Konda (no kernel restart needed!)
    print("\n📦 Installing Konda...")
    !pip install -q konda
    import konda
    konda.install()

    # Accept conda terms of service (required before conda install)
    print("\n📋 Accepting conda terms of service...")
    !conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main 2>/dev/null || true
    !conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r 2>/dev/null || true

    # Create conda environment with Python 3.11 (AmberTools doesn't support 3.13)
    print("\n" + "="*60)
    print("🐍 Creating conda environment with Python 3.11...")
    print("(Colab uses Python 3.13, but AmberTools requires <=3.12)")
    print("="*60)
    !conda create -n mdzen python=3.11 -y 2>&1 | tail -5

    # Install conda packages in the mdzen environment
    print("\n" + "="*60)
    print("⚗️ Installing AmberTools + scientific packages...")
    print("This takes ~5-10 minutes. Please wait.")
    print("="*60)
    !conda install -n mdzen -y -c conda-forge ambertools=23 openmm rdkit pdbfixer 2>&1 | tail -10
    print(f"✓ Conda packages installed ({time.time() - start_time:.0f}s)")

    # Clone repository
    print("\n📥 Cloning mdzen repository...")
    !rm -rf /content/mdzen
    !git clone -q https://github.com/matsunagalab/mdzen.git /content/mdzen
    %cd /content/mdzen

    # Install pip dependencies (in Colab's Python for Gradio/ADK)
    print("\n📦 Installing Python dependencies...")
    !pip install -q gradio py3Dmol nest_asyncio matplotlib 2>&1 | tail -3
    print("✓ Gradio, py3Dmol installed")
    !pip install -q google-adk google-genai litellm 2>&1 | tail -3
    print("✓ Google ADK, LiteLLM installed")
    !pip install -q -e . 2>&1 | tail -3
    print("✓ mdzen installed")

    # Set environment variables for the mdzen conda environment
    os.environ["AMBERHOME"] = "/usr/local/envs/mdzen"
    os.environ["MDZEN_CONDA_ENV"] = "mdzen"
    
    # Add conda env bin to PATH for AmberTools commands
    conda_bin = "/usr/local/envs/mdzen/bin"
    os.environ["PATH"] = f"{conda_bin}:{os.environ['PATH']}"

    # Add paths
    sys.path.insert(0, '/content/mdzen/src')
    sys.path.insert(0, '/content/mdzen')

    total_time = time.time() - start_time
    print(f"\n" + "="*60)
    print(f"✅ Setup complete! ({total_time/60:.1f} minutes)")
    print(f"🤖 LLM Provider: {detected_provider.upper() if detected_provider else 'NOT SET'}")
    print(f"🧪 AMBERHOME: {os.environ['AMBERHOME']}")
    print("="*60)

else:
    # Local development - add src to path
    sys.path.insert(0, './src')
    sys.path.insert(0, '.')
    print("Local environment - dependencies should be pre-installed.")

---
## Phase 1: Clarification Chat

Describe your simulation and the AI agent will ask clarifying questions to generate a **SimulationBrief**.

**Example prompts:**
- "Setup MD for PDB 1AKE in water, 1 ns at 300K"
- "I want to simulate lysozyme (PDB 1LYZ) with explicit solvent"
- "Run a short simulation of insulin (PDB 4INS), chain A only"

After the brief is generated, proceed to the next cell to **review and edit** the parameters.

In [ ]:
import gradio as gr
import asyncio
import nest_asyncio
import json
from pathlib import Path

nest_asyncio.apply()

# ============================================================================
# Global State (shared across cells)
# ============================================================================
if 'mdzen_state' not in dir():
    mdzen_state = {
        "session_id": None,
        "session_service": None,
        "session_dir": None,
        "simulation_brief": None,
        "workflow_outputs": {},
    }

# ============================================================================
# Session Initialization
# ============================================================================
def init_session():
    """Create new session for MD workflow"""
    import random
    import string
    
    job_id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    
    IN_COLAB = 'google.colab' in sys.modules
    if IN_COLAB:
        base_dir = Path("/content/mdzen/outputs")
    else:
        base_dir = Path("./outputs")
    
    session_dir = base_dir / f"job_{job_id}"
    session_dir.mkdir(parents=True, exist_ok=True)
    
    mdzen_state["session_id"] = f"job_{job_id}"
    mdzen_state["session_dir"] = str(session_dir)
    
    return session_dir

# ============================================================================
# Phase 1: Clarification Chat
# ============================================================================
def phase1_chat(message, history):
    """Phase 1: Gather requirements and generate SimulationBrief"""
    import traceback
    try:
        loop = asyncio.get_event_loop()
        
        # Initialize session if needed
        if mdzen_state["session_dir"] is None:
            init_session()
        
        # Lazy import agents
        from mdzen.agents.clarification_agent import create_clarification_agent
        from google.adk.runners import Runner
        from google.genai import types
        from mdzen.state.session_manager import (
            create_session_service,
            initialize_session_state,
            get_session_state,
        )
        
        # Create session service if not exists
        if mdzen_state["session_service"] is None:
            db_path = Path(mdzen_state["session_dir"]) / "session.db"
            mdzen_state["session_service"] = create_session_service(str(db_path), in_memory=False)
            
            # Initialize session state
            loop.run_until_complete(initialize_session_state(
                session_service=mdzen_state["session_service"],
                app_name="mdzen",
                user_id="default",
                session_id=mdzen_state["session_id"],
                session_dir=mdzen_state["session_dir"],
            ))
        
        # Create clarification agent
        agent, toolsets = create_clarification_agent()
        
        # Run agent
        runner = Runner(
            app_name="mdzen",
            agent=agent,
            session_service=mdzen_state["session_service"],
        )
        
        user_message = types.Content(
            role="user",
            parts=[types.Part(text=message)],
        )
        
        response_text = ""
        
        async def run_agent():
            nonlocal response_text
            async for event in runner.run_async(
                user_id="default",
                session_id=mdzen_state["session_id"],
                new_message=user_message,
            ):
                if event.is_final_response() and event.content:
                    if hasattr(event.content, 'parts'):
                        for part in event.content.parts:
                            if hasattr(part, 'text'):
                                response_text += part.text
                    else:
                        response_text = str(event.content)
        
        loop.run_until_complete(run_agent())
        
        # Check for SimulationBrief in state
        state = loop.run_until_complete(get_session_state(
            mdzen_state["session_service"],
            "mdzen",
            "default",
            mdzen_state["session_id"]
        ))
        
        if state and state.get("simulation_brief"):
            mdzen_state["simulation_brief"] = state["simulation_brief"]
            brief = mdzen_state["simulation_brief"]
            if isinstance(brief, dict):
                response_text += f"\n\n---\n✅ **SimulationBrief Generated!**\n"
                response_text += f"- PDB: {brief.get('pdb_id', 'N/A')}\n"
                response_text += f"- Temperature: {brief.get('temperature', 300)}K\n"
                response_text += f"- Simulation Time: {brief.get('simulation_time_ns', 1.0)}ns\n"
                response_text += f"\n**→ Run the next cell to review and edit the brief.**"
        
        yield response_text if response_text else "Processing..."
        
        # Cleanup toolsets
        for toolset in toolsets:
            loop.run_until_complete(toolset.close())
    
    except Exception as e:
        yield f"Error: {e}\n\n{traceback.format_exc()}"

# ============================================================================
# Launch Phase 1 Interface
# ============================================================================
with gr.Blocks(title="Phase 1: Clarification", theme=gr.themes.Soft()) as phase1_demo:
    gr.Markdown("## 🗣️ Phase 1: Describe Your Simulation")
    gr.ChatInterface(
        fn=phase1_chat,
        examples=[
            "Setup MD for PDB 1AKE in water, 1 ns at 300K",
            "Simulate lysozyme (PDB 1LYZ) with explicit solvent",
            "Run a short simulation of insulin (PDB 4INS), chain A only",
        ],
        retry_btn=None,
        undo_btn=None,
    )

phase1_demo.launch(share=True, debug=True)

---
## SimulationBrief Editor

Review and edit the simulation parameters before executing the workflow.

**Fields:**
- **Structure**: PDB ID, AlphaFold ID, or FASTA sequence
- **Simulation**: Temperature, pressure, simulation time
- **Solvation**: Water model, box padding, salt concentration
- **Options**: Chain selection, force field

Click **"Save Brief"** when done, then proceed to Phase 2.

In [ ]:
import gradio as gr

# ============================================================================
# SimulationBrief Editor
# ============================================================================

def load_brief():
    """Load current brief from state"""
    brief = mdzen_state.get("simulation_brief", {})
    if isinstance(brief, str):
        import json
        brief = json.loads(brief)
    return (
        brief.get("pdb_id", ""),
        brief.get("alphafold_id", ""),
        brief.get("fasta_sequence", ""),
        brief.get("ligand_smiles", ""),
        brief.get("select_chains", ""),
        brief.get("temperature", 300),
        brief.get("pressure_bar", 1.0),
        brief.get("simulation_time_ns", 1.0),
        brief.get("water_model", "tip3p"),
        brief.get("box_padding", 12.0),
        brief.get("salt_concentration", 0.15),
        brief.get("force_field", "ff19SB"),
    )

def save_brief(pdb_id, alphafold_id, fasta_sequence, ligand_smiles, select_chains,
               temperature, pressure_bar, simulation_time_ns, water_model,
               box_padding, salt_concentration, force_field):
    """Save edited brief to state"""
    brief = {
        "pdb_id": pdb_id if pdb_id else None,
        "alphafold_id": alphafold_id if alphafold_id else None,
        "fasta_sequence": fasta_sequence if fasta_sequence else None,
        "ligand_smiles": ligand_smiles if ligand_smiles else None,
        "select_chains": select_chains if select_chains else None,
        "temperature": float(temperature),
        "pressure_bar": float(pressure_bar),
        "simulation_time_ns": float(simulation_time_ns),
        "water_model": water_model,
        "box_padding": float(box_padding),
        "salt_concentration": float(salt_concentration),
        "force_field": force_field,
        "ensemble": "NPT",
        "cubic_box": True,
        "ph": 7.4,
    }
    mdzen_state["simulation_brief"] = brief
    return f"✅ Brief saved!\n\n```json\n{json.dumps(brief, indent=2)}\n```\n\n**→ Run the next cell to execute the workflow.**"

# ============================================================================
# Build Editor Interface
# ============================================================================
with gr.Blocks(title="Brief Editor", theme=gr.themes.Soft()) as editor_demo:
    gr.Markdown("## ✏️ Edit SimulationBrief")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Structure")
            pdb_id = gr.Textbox(label="PDB ID", placeholder="e.g., 1AKE")
            alphafold_id = gr.Textbox(label="AlphaFold ID", placeholder="e.g., AF-P00533-F1")
            fasta_sequence = gr.Textbox(label="FASTA Sequence", placeholder="MVLSPADKTN...", lines=3)
            ligand_smiles = gr.Textbox(label="Ligand SMILES", placeholder="e.g., CC(=O)Oc1ccccc1C(=O)O")
            select_chains = gr.Textbox(label="Select Chains", placeholder="e.g., A,B or leave empty for all")
        
        with gr.Column():
            gr.Markdown("### Simulation Parameters")
            temperature = gr.Slider(label="Temperature (K)", minimum=250, maximum=400, value=300, step=5)
            pressure_bar = gr.Number(label="Pressure (bar)", value=1.0)
            simulation_time_ns = gr.Slider(label="Simulation Time (ns)", minimum=0.01, maximum=100, value=1.0, step=0.1)
            
            gr.Markdown("### Solvation")
            water_model = gr.Dropdown(label="Water Model", choices=["tip3p", "tip4pew", "opc", "spce"], value="tip3p")
            box_padding = gr.Slider(label="Box Padding (Å)", minimum=8, maximum=20, value=12, step=1)
            salt_concentration = gr.Slider(label="Salt Concentration (M)", minimum=0, maximum=0.5, value=0.15, step=0.01)
            force_field = gr.Dropdown(label="Force Field", choices=["ff19SB", "ff14SB", "ff99SB"], value="ff19SB")
    
    with gr.Row():
        load_btn = gr.Button("📥 Load from Phase 1", variant="secondary")
        save_btn = gr.Button("💾 Save Brief", variant="primary")
    
    output = gr.Markdown()
    
    load_btn.click(
        load_brief,
        outputs=[pdb_id, alphafold_id, fasta_sequence, ligand_smiles, select_chains,
                 temperature, pressure_bar, simulation_time_ns, water_model,
                 box_padding, salt_concentration, force_field]
    )
    
    save_btn.click(
        save_brief,
        inputs=[pdb_id, alphafold_id, fasta_sequence, ligand_smiles, select_chains,
                temperature, pressure_bar, simulation_time_ns, water_model,
                box_padding, salt_concentration, force_field],
        outputs=output
    )

editor_demo.launch(share=True, debug=True)

---
## Phase 2: Execute Workflow

Execute the MD workflow step by step:

1. **prepare_complex** - Fetch structure and parameterize ligands
2. **solvate** - Add water box and ions
3. **build_topology** - Generate Amber topology files
4. **run_simulation** - Run MD with OpenMM

Click each button to execute that step. Progress is shown below.

In [ ]:
import gradio as gr
import traceback
from pathlib import Path

# ============================================================================
# Workflow Step Functions
# ============================================================================
def run_prepare_complex():
    """Step 1: Fetch and prepare structure"""
    try:
        brief = mdzen_state.get("simulation_brief")
        if not brief:
            return "❌ No SimulationBrief found. Run Phase 1 first."
        
        session_dir = Path(mdzen_state["session_dir"])
        
        import importlib
        import servers.structure_server as structure_module
        importlib.reload(structure_module)
        
        pdb_id = brief.get('pdb_id')
        if not pdb_id:
            return "❌ No PDB ID specified in SimulationBrief"
        
        # Fetch structure
        import asyncio
        loop = asyncio.get_event_loop()
        fetch_result = loop.run_until_complete(structure_module.fetch_molecules(
            pdb_id=pdb_id,
            source="pdb",
            prefer_format="pdb",
            output_dir=str(session_dir)
        ))
        
        if not fetch_result["success"]:
            return f"❌ Fetch failed: {fetch_result.get('errors')}"
        
        structure_file = fetch_result["file_path"]
        
        # Prepare complex
        complex_result = structure_module.prepare_complex(
            structure_file=structure_file,
            select_chains=brief.get('select_chains'),
            ph=brief.get('ph', 7.4),
            process_proteins=True,
            process_ligands=True,
            run_parameterization=True,
            output_dir=str(session_dir)
        )
        
        if not complex_result["success"]:
            return f"❌ Prepare failed: {complex_result.get('errors')}"
        
        mdzen_state["workflow_outputs"]["structure_file"] = structure_file
        mdzen_state["workflow_outputs"]["merged_pdb"] = complex_result["merged_pdb"]
        mdzen_state["workflow_outputs"]["complex_result"] = complex_result
        
        return f"✅ **prepare_complex complete!**\n\n- Fetched: {Path(structure_file).name}\n- Proteins: {len(complex_result['proteins'])}\n- Ligands: {len(complex_result['ligands'])}\n- Output: {Path(complex_result['merged_pdb']).name}"
    
    except Exception as e:
        return f"❌ Error: {e}\n\n{traceback.format_exc()}"

def run_solvate():
    """Step 2: Solvate structure"""
    try:
        brief = mdzen_state.get("simulation_brief")
        session_dir = Path(mdzen_state["session_dir"])
        
        merged_pdb = mdzen_state["workflow_outputs"].get("merged_pdb")
        if not merged_pdb:
            return "❌ No merged_pdb. Run prepare_complex first."
        
        import importlib
        import servers.solvation_server as solvation_module
        importlib.reload(solvation_module)
        
        solvate_result = solvation_module.solvate_structure(
            pdb_file=str(Path(merged_pdb).resolve()),
            output_dir=str(session_dir),
            output_name="solvated",
            dist=brief.get('box_padding', 12.0),
            cubic=brief.get('cubic_box', True),
            salt=True,
            saltcon=brief.get('salt_concentration', 0.15)
        )
        
        if not solvate_result["success"]:
            return f"❌ Solvate failed: {solvate_result.get('errors')}"
        
        mdzen_state["workflow_outputs"]["solvated_pdb"] = solvate_result["output_file"]
        mdzen_state["workflow_outputs"]["box_dimensions"] = solvate_result.get("box_dimensions")
        
        stats = solvate_result.get('statistics', {})
        return f"✅ **solvate complete!**\n\n- Total atoms: {stats.get('total_atoms', '?')}\n- Water molecules: {stats.get('water_molecules', '?')}\n- Output: {Path(solvate_result['output_file']).name}"
    
    except Exception as e:
        return f"❌ Error: {e}\n\n{traceback.format_exc()}"

def run_build_topology():
    """Step 3: Build Amber topology"""
    try:
        brief = mdzen_state.get("simulation_brief")
        session_dir = Path(mdzen_state["session_dir"])
        
        solvated_pdb = mdzen_state["workflow_outputs"].get("solvated_pdb")
        if not solvated_pdb:
            return "❌ No solvated_pdb. Run solvate first."
        
        import importlib
        import servers.amber_server as amber_module
        importlib.reload(amber_module)
        
        # Get ligand parameters if any
        ligand_params = []
        complex_result = mdzen_state["workflow_outputs"].get("complex_result", {})
        for lig in complex_result.get("ligands", []):
            if lig.get("success") and lig.get("mol2_file"):
                ligand_params.append({
                    "mol2": lig["mol2_file"],
                    "frcmod": lig["frcmod_file"],
                    "residue_name": lig["ligand_id"][:3].upper()
                })
        
        amber_result = amber_module.build_amber_system(
            pdb_file=solvated_pdb,
            ligand_params=ligand_params if ligand_params else None,
            box_dimensions=mdzen_state["workflow_outputs"].get("box_dimensions"),
            water_model=brief.get('water_model', 'tip3p'),
            output_name="system",
            output_dir=str(session_dir)
        )
        
        if not amber_result['success']:
            return f"❌ Build failed: {amber_result.get('errors')}"
        
        mdzen_state["workflow_outputs"]["parm7"] = amber_result['parm7']
        mdzen_state["workflow_outputs"]["rst7"] = amber_result['rst7']
        
        return f"✅ **build_topology complete!**\n\n- Topology: {Path(amber_result['parm7']).name}\n- Coordinates: {Path(amber_result['rst7']).name}"
    
    except Exception as e:
        return f"❌ Error: {e}\n\n{traceback.format_exc()}"

def run_simulation():
    """Step 4: Run MD simulation with OpenMM"""
    try:
        brief = mdzen_state.get("simulation_brief")
        session_dir = Path(mdzen_state["session_dir"])
        
        parm7_file = mdzen_state["workflow_outputs"].get("parm7")
        rst7_file = mdzen_state["workflow_outputs"].get("rst7")
        
        if not parm7_file or not rst7_file:
            return "❌ No topology files. Run build_topology first."
        
        import openmm as mm
        from openmm import app, unit
        from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, DCDReporter, PDBFile
        
        # Select platform
        platform = None
        platform_name = "CPU"
        for name in ['CUDA', 'OpenCL', 'CPU']:
            try:
                platform = mm.Platform.getPlatformByName(name)
                platform_name = name
                break
            except:
                continue
        
        # Load topology
        prmtop = AmberPrmtopFile(parm7_file)
        inpcrd = AmberInpcrdFile(rst7_file)
        
        # Simulation parameters
        temperature = brief.get('temperature', 300.0) * unit.kelvin
        pressure = (brief.get('pressure_bar') or 1.0) * unit.atmosphere
        timestep = 2.0 * unit.femtoseconds
        sim_time = brief.get('simulation_time_ns', 0.1)
        
        # Create system
        system = prmtop.createSystem(
            nonbondedMethod=app.PME,
            nonbondedCutoff=10 * unit.angstrom,
            constraints=app.HBonds,
            rigidWater=True
        )
        system.addForce(mm.MonteCarloBarostat(pressure, temperature, 25))
        
        # Create simulation
        integrator = mm.LangevinMiddleIntegrator(temperature, 1/unit.picosecond, timestep)
        simulation = Simulation(prmtop.topology, system, integrator, platform)
        simulation.context.setPositions(inpcrd.positions)
        if inpcrd.boxVectors:
            simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
        
        # Minimize
        simulation.minimizeEnergy(maxIterations=500)
        simulation.context.setVelocitiesToTemperature(temperature)
        
        # Setup output
        md_dir = session_dir / "md_simulation"
        md_dir.mkdir(exist_ok=True)
        
        dcd_file = md_dir / "trajectory.dcd"
        total_steps = int(sim_time * 1e6 / 2)
        report_interval = max(100, total_steps // 100)
        simulation.reporters.append(DCDReporter(str(dcd_file), report_interval))
        
        # Run
        simulation.step(total_steps)
        
        # Save final state
        final_pdb = md_dir / "final_state.pdb"
        state = simulation.context.getState(getPositions=True)
        with open(final_pdb, 'w') as f:
            PDBFile.writeFile(simulation.topology, state.getPositions(), f)
        
        mdzen_state["workflow_outputs"]["trajectory"] = str(dcd_file)
        mdzen_state["workflow_outputs"]["final_pdb"] = str(final_pdb)
        
        return f"✅ **run_simulation complete!**\n\n- Platform: {platform_name}\n- Simulation time: {sim_time} ns\n- Trajectory: {dcd_file.name}\n- Final structure: {final_pdb.name}\n\n**→ Run the next cell for visualization.**"
    
    except Exception as e:
        return f"❌ Error: {e}\n\n{traceback.format_exc()}"

# ============================================================================
# Build Phase 2 Interface
# ============================================================================
with gr.Blocks(title="Phase 2: Execute", theme=gr.themes.Soft()) as phase2_demo:
    gr.Markdown("## ⚙️ Phase 2: Execute Workflow")
    
    with gr.Row():
        btn1 = gr.Button("1️⃣ prepare_complex", variant="primary")
        btn2 = gr.Button("2️⃣ solvate", variant="primary")
        btn3 = gr.Button("3️⃣ build_topology", variant="primary")
        btn4 = gr.Button("4️⃣ run_simulation", variant="primary")
    
    output = gr.Markdown(value="Click a button to start...")
    
    btn1.click(run_prepare_complex, outputs=output)
    btn2.click(run_solvate, outputs=output)
    btn3.click(run_build_topology, outputs=output)
    btn4.click(run_simulation, outputs=output)

phase2_demo.launch(share=True, debug=True)

---
## Visualization

View your trajectory animation with py3Dmol. Click the button to load and visualize the simulation results.

In [ ]:
import py3Dmol
import numpy as np
import tempfile
from pathlib import Path

def visualize_trajectory():
    """Create py3Dmol visualization of trajectory"""
    try:
        if not mdzen_state.get("workflow_outputs") or 'trajectory' not in mdzen_state["workflow_outputs"]:
            print("❌ No trajectory available. Complete the workflow first.")
            return None
        
        import mdtraj as md
        
        traj_file = mdzen_state["workflow_outputs"]['trajectory']
        top_file = mdzen_state["workflow_outputs"]['parm7']
        
        print(f"📂 Loading trajectory: {traj_file}")
        traj = md.load(traj_file, top=top_file)
        print(f"✓ Loaded {traj.n_frames} frames, {traj.n_atoms} atoms")
        
        # Select protein only
        protein_indices = traj.topology.select('protein')
        if len(protein_indices) == 0:
            print("❌ No protein atoms found")
            return None
        
        traj_protein = traj.atom_slice(protein_indices)
        print(f"✓ Selected {len(protein_indices)} protein atoms")
        
        # Sample frames
        max_frames = 20
        if traj_protein.n_frames > max_frames:
            frame_indices = np.linspace(0, traj_protein.n_frames - 1, max_frames, dtype=int)
            traj_viz = traj_protein[frame_indices]
        else:
            traj_viz = traj_protein
        
        # Write multi-model PDB
        with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
            tmp_path = tmp.name
        
        with open(tmp_path, 'w') as f:
            for i in range(traj_viz.n_frames):
                frame_tmp = tmp_path + f".frame{i}.pdb"
                traj_viz[i].save_pdb(frame_tmp, force_overwrite=True)
                with open(frame_tmp, 'r') as ff:
                    content = ff.read()
                f.write(f"MODEL     {i + 1}\n")
                for line in content.split('\n'):
                    if not line.startswith('MODEL') and not line.startswith('ENDMDL') and line.strip():
                        f.write(line + '\n')
                f.write("ENDMDL\n")
                Path(frame_tmp).unlink()
        
        with open(tmp_path, 'r') as f:
            traj_pdb = f.read()
        Path(tmp_path).unlink()
        
        # Create 3D view
        view = py3Dmol.view(width=800, height=500)
        view.addModelsAsFrames(traj_pdb, 'pdb')
        view.setStyle({'cartoon': {'color': 'spectrum'}})
        view.zoomTo()
        view.animate({'loop': 'forward', 'reps': 0, 'interval': 100})
        
        print(f"\n🎬 Trajectory Animation: {traj_viz.n_frames} frames")
        print(f"⏱️ Total time: {traj.time[-1]:.1f} ps")
        
        return view.show()
    
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run visualization
visualize_trajectory()

---
## Download Results

In [ ]:
import sys
from pathlib import Path

IN_COLAB = 'google.colab' in sys.modules

if mdzen_state.get("session_dir"):
    session_dir = Path(mdzen_state["session_dir"])
    
    if session_dir.exists():
        print(f"📂 Session directory: {session_dir}")
        print("\n📄 Generated files:")
        for f in sorted(session_dir.rglob('*')):
            if f.is_file():
                rel_path = f.relative_to(session_dir)
                size_kb = f.stat().st_size / 1024
                print(f"  {rel_path} ({size_kb:.1f} KB)")
        
        if IN_COLAB:
            from google.colab import files
            import shutil
            
            zip_name = f"{session_dir.name}.zip"
            shutil.make_archive(str(session_dir), 'zip', session_dir)
            print(f"\n⬇️ Downloading {zip_name}...")
            files.download(f"{session_dir}.zip")
        else:
            print(f"\n📁 Files are in: {session_dir}")
    else:
        print("❌ Session directory not found.")
else:
    print("❌ No session available. Run the workflow first.")

---

## Next Steps

1. **Longer simulations**: Modify the simulation time in your Phase 1 request
2. **Analysis**: Use MDTraj for RMSD, RMSF, hydrogen bonds, etc.
3. **Different systems**: Try membrane proteins, protein-ligand complexes
4. **Command line**: Use `main.py run` for local development

For more information, see the [GitHub repository](https://github.com/matsunagalab/mdzen).